In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

%matplotlib notebook

In [ ]:
file_path = "../data/joints_calibration_analysis_data/body.h5"
with h5py.File(file_path, "r") as f:
    device_keys = sorted([k for k in f.keys() if k.startswith("device_")])
    for device_key in device_keys:
        grp = f[device_key]
        print(grp["ts"][:])
        print(grp["frame_idx"][:])

In [ ]:
with h5py.File(file_path, "r") as f:
    grp = f["device_0"]
    idx = np.where(grp["frame_idx"][:] == 537)
    print(idx)
    print(grp["ts"][idx])
    print(grp["frame_idx"][idx])

In [ ]:
cmap = plt.get_cmap("tab20")

def plot_joints_matplotlib(file_path, frame_idx):
    fig = plt.figure()
    ax = fig.add_subplot(projection="3d")

    with h5py.File(file_path, "r") as f:
        connections = f["joint_connections"][:]
        device_keys = sorted([k for k in f.keys() if k.startswith("device_")])
        for i, device_key in enumerate(device_keys):
            device_grp = f[device_key]
            bodies_keys = sorted([k for k in device_grp.keys() if k.startswith("body_")])
            for j, body_key in enumerate(bodies_keys):
                if frame_idx >= grp["positions"].shape[0]:
                    continue

                pts = grp["positions"][frame_idx]
                ax.scatter(pts[:, 0], pts[:, 1], pts[:, 2], color=colors[i % 3], label=f"Device {device_key}")
                for c in connections:
                    ax.plot(pts[c, 0], pts[c, 1], pts[c, 2], color=colors[i % 3], alpha=0.3)
    ax.set_xlabel("x"); ax.set_ylabel("y"); ax.set_zlabel("z")
    ax.legend(loc="upper right")

    plt.show()

In [ ]:
plot_joints_matplotlib("../data/joints_calibration_analysis_data/body.h5", 500)